# Step-by-step guide of GitHub API researching

## 1. Install required libraries




In [ ]:
!pip install requests

## 2. Set up basic imports

In [ ]:
import requests
import json
import time
from pprint import pprint

## 3. Set up token and headers

In [ ]:
TOKEN = "your-token"
HEADERS = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {TOKEN}",
    "X-GitHub-Api-Version": "2022-11-28"
}

## 4. Handle troubles

### Handle authentication

In [ ]:
def test_authentication():
    url = "https://api.github.com/user"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        print("Authentication successful!")
    else:
        print(f"Authentication failed: {response.status_code}")
        pprint(response.json())

test_authentication()

### - Handle rate limits

In [ ]:
def check_rate_limit():
    url = "https://api.github.com/rate_limit"
    response = requests.get(url, headers=HEADERS)
    rate_limit = response.json()
    remaining = rate_limit['rate']['remaining']
    reset_time = rate_limit['rate']['reset']

    if remaining == 0:
        wait_time = reset_time - time.time()
        print(f"Rate limit exceeded. Waiting for {wait_time:.2f} seconds.")
        time.sleep(wait_time + 1)
    else:
        print(f"Requests remaining: {remaining}")

### - Handle pagination

In [ ]:
def fetch_all_pages(fetch_function, *args, **kwargs):
    all_results = []
    page = 1
    while True:
        print(f"Fetching page {page}...")
        check_rate_limit()
        results = fetch_function(*args, **kwargs, page=page)
        if not results:
            print("No more results to fetch.")
            break
        all_results.extend(results)
        page += 1
        time.sleep(0.5)
    return all_results

## 5. Define API functions

### - Search repositories

In [ ]:
def search_repositories(query, per_page=10, page=1):
    url = "https://api.github.com/search/repositories"
    params = {"q": query, "per_page": per_page, "page": page}
    response = requests.get(url, headers=HEADERS, params=params)
    if response.status_code == 200:
        return response.json().get("items", [])
    else:
        print(f"Error: {response.status_code}")
        pprint(response.json())
        return []

### - Fetch commits

In [ ]:
def get_commits(owner, repo, per_page=10, page=1):
    url = f"https://api.github.com/repos/{owner}/{repo}/commits"
    params = {"per_page": per_page, "page": page}
    response = requests.get(url, headers=HEADERS, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        pprint(response.json())
        return []

### - Retrieve repository contents

In [ ]:
def get_contents(owner, repo, path=""):
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{path}"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        pprint(response.json())
        return []

### - Save data to JSON-file

In [ ]:
def save_to_json(data, filename):
    with open(filename, "w") as file:
        json.dump(data, file, indent=4, ensure_ascii=False)
    print(f"Data saved to {filename}")

## 6. Extract data

### - Fetch all repositories

In [ ]:
all_repositories = search_repositories(query="python", per_page=5)
print(f"Total repositories fetched: {len(all_repositories)}")
save_to_json(all_repositories, "repositories.json")

### - Fetch all commits

In [ ]:
all_commits = fetch_all_pages(get_commits, owner="octocat", repo="Hello-World", per_page=5)
print(f"Total commits fetched: {len(all_commits)}")
save_to_json(all_commits, "commits.json")

### - Fetch all repository contents

In [ ]:
all_contents = get_contents(owner="Timons172", repo="Library-Management-System")
print(f"Total contents fetched: {len(all_contents)}")
save_to_json(all_contents, "contents.json")